# 1. Configurar Ambiente

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
llm = dbutils.widgets.get("llm")
vs_index = dbutils.widgets.get("index")
vs_endpoint = dbutils.widgets.get("vs_endpoint")

In [0]:
import dspy

# Configurar LLM que usa DSPy
lm = dspy.LM(llm)
dspy.configure(lm=lm)

import mlflow

# Configurar MLflow con Databricks
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Workspace/Shared/MLflow Experiments/DSPy RAG")
mlflow.dspy.autolog(log_compiles=True, log_evals=True)

# 2. Configurar RAG

# 2.1 Definir Signature

In [0]:
class RAG_SIGNATURE(dspy.Signature):
  """Receive user's request, search for context in index, and recommend a cooking recipe."""
  question: str = dspy.InputField(desc="Customer question on cooking recipe.")
  context: str = dspy.InputField(desc="Vector Search Index of cooking recipes.")
  recommendation: str = dspy.OutputField(desc="Recommended cooking recipe.")

# 2.2 Definir Lógica de RAG

In [0]:
class Recommendation(dspy.Module):
  def __init__(self, for_mosaic_agent=True):
    from databricks.vector_search.client import VectorSearchClient
    from databricks.vector_search.utils import CredentialStrategy
    self.response_generator = dspy.Predict(RAG_SIGNATURE)

    # Definir cliente de Vector Search
    client = VectorSearchClient(
      credential_strategy=CredentialStrategy.MODEL_SERVING_USER_CREDENTIALS,
      disable_notice=True
    )
    self.retriever = client.get_index(
        endpoint_name=vs_endpoint, 
        index_name=f"{catalog}.{schema}.{vs_index}"
    )

  # Definir función de búsqueda
  def documentation_vector_search(self, text):
      # Obtener contexto usando búsqueda por similitud
      retrieved_context = self.retriever.similarity_search(
          query_text=text,
          columns=["title", "ingredients", "directions"],
          num_results=3,
          disable_notice=True
      )
      return(retrieved_context)
  
  # Generar y devolver respuesta
  def forward(self, question):
    context = self.documentation_vector_search(question)
    response = self.response_generator(context=context, question=question)
    return response

# 3. Probar agente

In [0]:
with mlflow.start_run(run_name="recommender"):
    recommender = Recommendation()
    print(recommender("What recipes can I prepare with chicken and rice?").recommendation)